In [6]:
# %% [code]
import pickle
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import asyncio

num_person = "both"

pkl_location_h1h2 = f'/Users/emilydu/Code/Code_lingustic/Data/f0_h1h2_xy_pkl_save/{num_person}/h1h2_xy.pkl'
pkl_location_f0   = f'/Users/emilydu/Code/Code_lingustic/Data/f0_h1h2_xy_pkl_save/{num_person}/f0_xy.pkl'

with open(pkl_location_h1h2, 'rb') as f:
    h1h2_dict = pickle.load(f)
with open(pkl_location_f0, 'rb') as f:
    f0_dict = pickle.load(f)

tones = list(h1h2_dict.keys())

tone_widget = widgets.Dropdown(options=tones, value=tones[0], description="Tone:")

initial_length = len(h1h2_dict[tone_widget.value]['x'])
frame_slider = widgets.IntSlider(min=0, max=initial_length - 1, step=1, value=0, description="Frame:")

h1h2_meter = widgets.FloatProgress(value=0, min=0, max=10, description="h1h2:")
f0_meter = widgets.FloatProgress(value=0, min=0, max=10, description="f0:")

h1h2_value_label = widgets.Label(value="h1h2 current: 0.00")
f0_value_label = widgets.Label(value="f0 current: 0.00")

y_span = [-10, 10]

fig = go.FigureWidget()
fig.add_scatter(x=[0, 0], y=y_span, mode='lines',
                line=dict(color='blue', width=4), showlegend=False)
fig.add_scatter(x=[0, 0], y=y_span, mode='lines',
                line=dict(color='red', width=4), showlegend=False)

fig.update_layout(title=f"Vocal Cord Animation: Tone {tone_widget.value}",
                  xaxis=dict(visible=False),
                  yaxis=dict(range=y_span),
                  showlegend=False)

def f0_to_color(current_f0, f0_min, f0_max):
    """
    Map the current f0 value to a color between blue (low) and red (high).
    """
    if f0_max == f0_min:
        t = 0.5
    else:
        t = (current_f0 - f0_min) / (f0_max - f0_min)
    r = int(t * 255)
    g = 0
    b = int((1 - t) * 255)
    return f"rgb({r},{g},{b})"

def update_figure(*args):
    tone = tone_widget.value
    frame = frame_slider.value

    time_arr = np.array(h1h2_dict[tone]['x'])
    h1h2_values = np.array(h1h2_dict[tone]['y'])
    f0_values = np.array(f0_dict[tone]['y'])
    
    frame = min(frame, len(time_arr) - 1, len(h1h2_values) - 1, len(f0_values) - 1)
    current_time = time_arr[frame]
    current_h1h2 = h1h2_values[frame]
    current_f0 = f0_values[frame]
    
    separation = current_h1h2 if current_h1h2 > 0 else 0.0
    left_x = -separation / 2
    right_x = separation / 2

    f0_min = float(np.min(f0_values))
    f0_max = float(np.max(f0_values))
    color = f0_to_color(current_f0, f0_min, f0_max)

    with fig.batch_update():
        fig.data[0].x = [left_x, left_x]
        fig.data[0].y = y_span
        fig.data[1].x = [right_x, right_x]
        fig.data[1].y = y_span
        fig.data[0].line.color = color
        fig.data[1].line.color = color
        fig.layout.title = f"Tone: {tone} | Frame: {frame} | Time: {current_time:.2f}"
    
    h1h2_meter.value = current_h1h2
    f0_meter.value = current_f0
    h1h2_value_label.value = f"h1h2 current: {current_h1h2:.2f}"
    f0_value_label.value = f"f0 current: {current_f0:.2f}"

def update_frame_slider_range(*args):
    tone = tone_widget.value
    new_max = len(h1h2_dict[tone]['x']) - 1
    frame_slider.max = new_max
    frame_slider.value = 0

    h1h2_arr = np.array(h1h2_dict[tone]['y'])
    f0_arr = np.array(f0_dict[tone]['y'])
    
    h1h2_meter.max = float(np.max(h1h2_arr))
    h1h2_meter.min = float(np.min(h1h2_arr))
    f0_meter.max = float(np.max(f0_arr))
    f0_meter.min = float(np.min(f0_arr))

start_button = widgets.Button(description="Start Animation", button_style='success')
stop_button = widgets.Button(description="Stop Animation", button_style='warning')
speed_slider = widgets.IntSlider(value=5, min=1, max=10, step=1, description="Speed:")

is_animating = False
animation_task = None

async def animate_frames():
    global is_animating
    is_animating = True
    total_frames = frame_slider.max + 1
    while is_animating:
        new_val = frame_slider.value + 1
        if new_val >= total_frames:
            new_val = 0
        frame_slider.value = new_val
        fps = (speed_slider.value / 10) * total_frames
        delay = 1 / fps
        await asyncio.sleep(delay)

def start_animation(b):
    global animation_task, is_animating
    if animation_task is None or animation_task.done():
        is_animating = True
        animation_task = asyncio.ensure_future(animate_frames())

def stop_animation(b):
    global is_animating, animation_task
    is_animating = False
    if animation_task is not None:
        animation_task.cancel()
        animation_task = None

start_button.on_click(start_animation)
stop_button.on_click(stop_animation)

# Set observers to update figure and meters on widget changes.
tone_widget.observe(update_frame_slider_range, names='value')
tone_widget.observe(update_figure, names='value')
frame_slider.observe(update_figure, names='value')

# Initial update.
update_frame_slider_range()
update_figure()

# Layout the UI.
ui_controls = widgets.HBox([tone_widget, frame_slider])
ui_meters = widgets.VBox([
    widgets.HBox([h1h2_meter, h1h2_value_label]),
    widgets.HBox([f0_meter, f0_value_label])
])
animation_controls = widgets.HBox([start_button, stop_button, speed_slider])

display(ui_controls, fig, ui_meters, animation_controls)

FigureWidget({
    'data': [{'line': {'color': 'rgb(246,0,8)', 'width': 4},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'uid': '8b3ac05e-31d0-4c77-9501-8c02c0498053',
              'x': [-3.3356666666666666, -3.3356666666666666],
              'y': [-10, 10]},
             {'line': {'color': 'rgb(246,0,8)', 'width': 4},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'uid': '827272fe-0efc-4708-ba63-260558951740',
              'x': [3.3356666666666666, 3.3356666666666666],
              'y': [-10, 10]}],
    'layout': {'showlegend': False,
               'template': '...',
               'title': {'text': 'Tone: qu | Frame: 0 | Time: 0.00'},
               'xaxis': {'visible': False},
               'yaxis': {'range': [-10, 10]}}
})

Python version: 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
numpy version: 1.26.4
plotly version: 6.0.0
ipywidgets version: 8.1.5
IPython version: 8.27.0
